# Import libraries and configuration

In [24]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('bmh')
from scipy.stats import linregress, t

filename = "yourfilenamehere.csv"
save_output = False

# Calibration Settings

In [25]:
# Settings
# Keep only the last n-injections
injections_to_keep = -3

# Insert here the standards (STD) you want to use to compute
# the calibration curve. At least two.
standards = ['std_name_1', 'std_name_2']

# Insert here the control standard name (SPY), if any
spy = ['spy_name']

# Column of interest for isotopic values (don't change)
d17O_column = 'd(17_16)Mean'
d18O_column = 'd(18_16)Mean'
dD_column = 'd(D_H)Mean'
E17_column = 'E17_Mean'

# TO REMOVE
STD_1 = 'LSCE'
STD_2 = 'DCS'
SPY   = 'VDC'



# Standard values

In [ ]:
# Standard values
from standards_values import standard_d17O
from standards_values import standard_d18O
from standards_values import standard_dD
print(standard_d17O)
print(standard_d18O)
print(standard_dD)

# Import data and first check

In [ ]:
df = pd.read_csv(filename, skipinitialspace=True)
#df.columns
mask = (df['Identifier 2'] == 'STD')
std_in_df = df[mask]['Identifier 1'].unique()

print("Check for standards:")
for std in standards:
    print(f"{std} in {filename}: {std in std_in_df}")
    
print("")
mask = (df['Identifier 2'] == 'SPY')
if sum(mask)>0:
    print("Check for control standard:")
    spy_in_df = df[mask]['Identifier 1'].unique()
    for spys in spy:
        print(f"{spys} in {filename}: {spys in spy_in_df}")

# Plot $H_{2}O$ to check injections

In [ ]:
fig, ax = plt.subplots(nrows=4, figsize = (5,5), dpi = 150)
ax[0].plot(df['H2O_Mean'], lw = .75)
ax[0].set_ylabel('$H_{2}O$ (ppm)')

ax[1].plot(df['d(17_16)Mean'], lw = .75)
ax[1].set_ylabel('$\delta^{17}O$ (‰)')

ax[2].plot(df['d(18_16)Mean'], lw = .75)
ax[2].set_ylabel('$\delta^{18}O$ (‰)')

ax[3].plot(df['d(D_H)Mean'], lw = .75)
ax[3].set_ylabel('$\delta D$ (‰)')
ax[3].set_xlabel('injection (#)')

# Compute regression curve for calibration

In [30]:
std_d17O_meas = np.array([])
std_d17O_true = np.array([])
std_d18O_meas = np.array([])
std_d18O_true = np.array([])
std_dD_meas = np.array([])
std_dD_true = np.array([])
for std in standards:
    mask = (df['Identifier 1'] == std) & (df['Identifier 2'] == 'STD')
    # Get positions in table
    positions = df[mask]['Analysis'].unique()
    buff_d17O = np.array([])
    buff_d18O = np.array([])
    buff_dD = np.array([])
    df_buff = df[mask]
    for position in positions:
        #print(position)
        mask_layer2 = df_buff['Analysis'] == position
        buff_d17O = np.append(buff_d17O, df_buff[mask_layer2][d17O_column].iloc[injections_to_keep:].mean())
        buff_d18O = np.append(buff_d18O, df_buff[mask_layer2][d18O_column].iloc[injections_to_keep:].mean())
        buff_dD = np.append(buff_dD, df_buff[mask_layer2][dD_column].iloc[injections_to_keep:].mean())
    std_d17O_meas = np.append(std_d17O_meas, buff_d17O.mean())
    std_d17O_true = np.append(std_d17O_true, standard_d17O[std])
    std_d18O_meas = np.append(std_d18O_meas, buff_d18O.mean())
    std_d18O_true = np.append(std_d18O_true, standard_d18O[std])
    std_dD_meas = np.append(std_dD_meas, buff_dD.mean())
    std_dD_true = np.append(std_dD_true, standard_dD[std])

# ----
mdl_d17O = linregress(std_d17O_meas, std_d17O_true)
mdl_d18O = linregress(std_d18O_meas, std_d18O_true)
mdl_dD = linregress(std_dD_meas, std_dD_true)

# Calibrate
df_calibrated = df.copy()
df_calibrated[d17O_column] = df[d17O_column]*mdl_d17O.slope+mdl_d17O.intercept
df_calibrated[d18O_column] = df[d18O_column]*mdl_d18O.slope+mdl_d18O.intercept
df_calibrated[dD_column] = df[dD_column]*mdl_dD.slope+mdl_dD.intercept
df_calibrated[E17_column] = 1e6*(np.log(df_calibrated[d17O_column]/1000  + 1) - 0.528*np.log(df_calibrated[d18O_column]/1000  + 1))

# Check control standard

In [ ]:
for spys in spy:
    print(spys+": -------------------")
    mask = (df_calibrated['Identifier 1'] == spys) & (df_calibrated['Identifier 2'] == 'SPY')
    analyses = df_calibrated[mask]['Analysis'].unique()
    buff = np.array([])
    print('d17O:-------------------')
    for analysis in analyses:
        mask = df_calibrated['Analysis'] == analysis
        print(f"{df_calibrated[mask][d17O_column].iloc[injections_to_keep:].mean():.4f}")
        buff = np.append(buff, df_calibrated[mask][d17O_column].iloc[injections_to_keep:].mean())
    print(f"Average: {buff.mean():.4f} ±{buff.std():.4f} ")
    
    buff = np.array([])
    print('d18O:-------------------')
    for analysis in analyses:
        mask = df_calibrated['Analysis'] == analysis
        print(f"{df_calibrated[mask][d18O_column].iloc[injections_to_keep:].mean():.4f}")
        buff = np.append(buff, df_calibrated[mask][d18O_column].iloc[injections_to_keep:].mean())
    print(f"Average: {buff.mean():.4f} ±{buff.std():.4f} ")
    
    buff = np.array([])
    print('dD:---------------------')
    for analysis in analyses:
        mask = df_calibrated['Analysis'] == analysis
        print(f"{df_calibrated[mask][dD_column].iloc[injections_to_keep:].mean():.2f}")
        buff = np.append(buff, df_calibrated[mask][dD_column].iloc[injections_to_keep:].mean())
    print(f"Average: {buff.mean():.2f} ±{buff.std():.2f} ")
        
    buff = np.array([])
    print('E17-O:-----------------')
    for analysis in analyses:
        mask = df_calibrated['Analysis'] == analysis
        print(f"{df_calibrated[mask][E17_column].iloc[injections_to_keep:].mean():.0f}")
        buff = np.append(buff, df_calibrated[mask][E17_column].iloc[injections_to_keep:].mean())
    print(f"Average: {buff.mean():.0f} ±{buff.std():.0f} ")
print('-----------------------')

In [ ]:
for std in standards:
    print(std+": -------------------")
    mask = (df_calibrated['Identifier 1'] == std) & (df_calibrated['Identifier 2'] == 'STD')
    analyses = df_calibrated[mask]['Analysis'].unique()
    buff = np.array([])
    print('d17O:-------------------')
    for analysis in analyses:
        mask = df_calibrated['Analysis'] == analysis
        print(f"{df_calibrated[mask][d17O_column].iloc[injections_to_keep:].mean():.4f}")
        buff = np.append(buff, df_calibrated[mask][d17O_column].iloc[injections_to_keep:].mean())
    print(f"Average: {buff.mean():.4f} ±{buff.std():.4f} ")
    
    buff = np.array([])
    print('d18O:-------------------')
    for analysis in analyses:
        mask = df_calibrated['Analysis'] == analysis
        print(f"{df_calibrated[mask][d18O_column].iloc[injections_to_keep:].mean():.4f}")
        buff = np.append(buff, df_calibrated[mask][d18O_column].iloc[injections_to_keep:].mean())
    print(f"Average: {buff.mean():.4f} ±{buff.std():.4f} ")
    
    buff = np.array([])
    print('dD:---------------------')
    for analysis in analyses:
        mask = df_calibrated['Analysis'] == analysis
        print(f"{df_calibrated[mask][dD_column].iloc[injections_to_keep:].mean():.2f}")
        buff = np.append(buff, df_calibrated[mask][dD_column].iloc[injections_to_keep:].mean())
    print(f"Average: {buff.mean():.2f} ±{buff.std():.2f} ")
        
    buff = np.array([])
    print('E17-O:-----------------')
    for analysis in analyses:
        mask = df_calibrated['Analysis'] == analysis
        print(f"{df_calibrated[mask][E17_column].iloc[injections_to_keep:].mean():.0f}")
        buff = np.append(buff, df_calibrated[mask][E17_column].iloc[injections_to_keep:].mean())
    print(f"Average: {buff.mean():.0f} ±{buff.std():.0f} ")
print('-----------------------')

# Plot results

In [ ]:
fig, ax = plt.subplots(nrows=4, figsize = (5,8), dpi = 300, sharex = True)

dummy_x = 0

for sample in df_calibrated[df_calibrated['Identifier 2'] == "SAMPLE"]['Identifier 1'].unique():
    mask = (df_calibrated['Identifier 1'] == sample)
    #print(sample)
    #print(df_calibrated[mask][d17O_column].iloc[injections_to_keep:].mean())
    ax[0].scatter(dummy_x, 
                  df_calibrated[mask][d17O_column].iloc[injections_to_keep:].mean(), s = 5, c = 'k')
    ax[0].text(dummy_x, 
                  df_calibrated[mask][d17O_column].iloc[injections_to_keep:].mean(), sample, size = 3)
    
    ax[1].scatter(dummy_x, 
                  df_calibrated[mask][d18O_column].iloc[injections_to_keep:].mean(), s = 8, c = 'k')
    ax[1].text(dummy_x, 
                  df_calibrated[mask][d18O_column].iloc[injections_to_keep:].mean(), sample, size = 3)
    
    ax[2].scatter(dummy_x, 
                  df_calibrated[mask][dD_column].iloc[injections_to_keep:].mean(), s = 8, c = 'k')
    ax[2].text(dummy_x, 
                  df_calibrated[mask][dD_column].iloc[injections_to_keep:].mean(), sample, size = 3)
    
    ax[3].scatter(dummy_x, 
                  df_calibrated[mask][E17_column].iloc[injections_to_keep:].mean(), s = 8, c = 'k')
    ax[3].text(dummy_x, 
                  df_calibrated[mask][E17_column].iloc[injections_to_keep:].mean(), sample, size = 3)
    
    dummy_x = dummy_x + 1



ax[0].set_ylabel('$\delta^{17}O$ (‰)')
ax[1].set_ylabel('$\delta^{18}O$ (‰)')
ax[2].set_ylabel('$\delta D$ (‰)')
ax[3].set_ylabel('$\Delta^{17}O$ (permeg)')

# Print and save data

In [ ]:
print("d17O")
for sample in df_calibrated[df_calibrated['Identifier 2'] == "SAMPLE"]['Identifier 1'].unique():
    mask = (df_calibrated['Identifier 1'] == sample)
    print(f"{sample}: {df_calibrated[mask][d17O_column].iloc[injections_to_keep:].mean():.4f} ± {df_calibrated[mask][d17O_column].iloc[injections_to_keep:].std():.4f}")

print("----------------")
print("d18O")
for sample in df_calibrated[df_calibrated['Identifier 2'] == "SAMPLE"]['Identifier 1'].unique():
    mask = (df_calibrated['Identifier 1'] == sample)
    print(f"{sample}: {df_calibrated[mask][d18O_column].iloc[injections_to_keep:].mean():.4f} ± {df_calibrated[mask][d18O_column].iloc[injections_to_keep:].std():.4f}")

print("----------------")
print("dD")
for sample in df_calibrated[df_calibrated['Identifier 2'] == "SAMPLE"]['Identifier 1'].unique():
    mask = (df_calibrated['Identifier 1'] == sample)
    print(f"{sample}: {df_calibrated[mask][dD_column].iloc[injections_to_keep:].mean():.2f} ± {df_calibrated[mask][dD_column].iloc[injections_to_keep:].std():.2f}")

print("----------------")
print("E17-O")
for sample in df_calibrated[df_calibrated['Identifier 2'] == "SAMPLE"]['Identifier 1'].unique():
    mask = (df_calibrated['Identifier 1'] == sample)
    print(f"{sample}: {df_calibrated[mask][E17_column].iloc[injections_to_keep:].mean():.0f} ± {df_calibrated[mask][E17_column].iloc[injections_to_keep:].std():.0f}")    
    

In [ ]:
if save_output:
    new_filename = filename[:-4] + "_calibrated.csv"
    print("data saved into: "+new_filename)